In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow import keras

In [2]:
print(tf.__version__)

2.1.0


In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images, train_labels), (test_images, test_labels)=fashion_mnist.load_data()
train_images.shape

(60000, 28, 28)

In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
len(train_images), len(test_images)

(60000, 10000)

In [8]:
print(train_images.shape, test_images.shape)

(60000, 28, 28) (10000, 28, 28)


In [9]:
train_images=train_images.reshape(len(train_images), 28, 28, 1)
test_images=test_images.reshape(len(test_images), 28, 28, 1)

print(train_images.shape, test_images.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


In [10]:
from keras.layers import Dense, Flatten, Activation, Dropout, Conv2D
from keras.activations import relu, sigmoid
from keras.models import Sequential, Model

Using TensorFlow backend.


In [11]:
def build_model(hp):
    model = keras.models.Sequential([
        
        keras.layers.Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=256, step=16), 
                        kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
                        activation='relu',
                        input_shape=(28, 28, 1)),
        
        keras.layers.Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16), 
                        kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
                        activation='relu'),

        keras.layers.Flatten(),

        keras.layers.Dense(units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16), 
                        activation='relu'),
        keras.layers.Dense(units=10, activation='softmax')
              
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 0.1])), 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy']
    )
              
    return model

In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [13]:
tuner_search=RandomSearch(build_model, 
                          objective='accuracy',
                          max_trials=1, 
                          directory='output',
                          project_name='MNIST_FASHION')

INFO:tensorflow:Reloading Oracle from existing project output\MNIST_FASHION\oracle.json
INFO:tensorflow:Reloading Tuner from output\MNIST_FASHION\tuner0.json


In [14]:
tuner_search.search(train_images, train_labels, epochs=2, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/2
 4032/54000 [=>............................] - ETA: 52:42 - loss: 2.2963 - accuracy: 0.093 - ETA: 28:55 - loss: 13.7087 - accuracy: 0.17 - ETA: 20:42 - loss: 11.4687 - accuracy: 0.16 - ETA: 16:27 - loss: 9.1600 - accuracy: 0.2031 - ETA: 13:59 - loss: 7.8055 - accuracy: 0.218 - ETA: 12:17 - loss: 6.9112 - accuracy: 0.208 - ETA: 11:02 - loss: 6.1950 - accuracy: 0.245 - ETA: 10:11 - loss: 5.6387 - accuracy: 0.273 - ETA: 9:28 - loss: 5.2274 - accuracy: 0.281 - ETA: 8:58 - loss: 4.8799 - accuracy: 0.28 - ETA: 8:32 - loss: 4.5620 - accuracy: 0.30 - ETA: 8:11 - loss: 4.2962 - accuracy: 0.30 - ETA: 7:52 - loss: 4.0673 - accuracy: 0.31 - ETA: 7:38 - loss: 3.9021 - accuracy: 0.32 - ETA: 7:25 - loss: 3.7149 - accuracy: 0.35 - ETA: 7:11 - loss: 3.5367 - accuracy: 0.36 - ETA: 7:03 - loss: 3.3754 - accuracy: 0.38 - ETA: 6:55 - loss: 3.2646 - accuracy: 0.40 - ETA: 6:49 - loss: 3.1555 - accuracy: 0.41 - ETA: 6:41 - loss: 3.0695 - accuracy: 0.4

KeyboardInterrupt: 

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]
model.summary()

In [ ]:
y_predicted=model.predict(test_images)
y_predicted[0]

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import pickle

In [ ]:
scores=model.evaluate(test_images, test_labels)
scores

In [ ]:
print(scores[1]*100)

In [ ]:
json_file=open('model.json', 'r')
loaded_model_json=json_file.read()
json_file.close()

In [ ]:
loaded_model=keras.models.model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")

In [ ]:
loaded_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
score=loaded_model.evaluate(test_images, test_labels)
score[1]*100